<a href="https://colab.research.google.com/github/sgokulakannankg19/ClinicalTrialFiles/blob/main/criteria_check_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import requests
from collections import Counter


import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
BASE_URL = "https://clinicaltrials.gov/api/query/study_fields?expr="
expression = "cancer"
format = "json"
fields = "NCTId"
final_url = BASE_URL + f"{expression}&fields={fields}&min_rnk=1&max_rnk=&fmt={format}" 

In [ ]:
final_url

'https://clinicaltrials.gov/api/query/study_fields?expr=cancer&fields=NCTId&min_rnk=1&max_rnk=&fmt=json'

In [ ]:
response = requests.get(final_url)
response.content

b'{\n  "StudyFieldsResponse":{\n    "APIVrs":"1.01.05",\n    "DataVrs":"2022:01:31 23:02:41.433",\n    "Expression":"cancer",\n    "NStudiesAvail":402977,\n    "NStudiesFound":98915,\n    "MinRank":1,\n    "MaxRank":20,\n    "NStudiesReturned":20,\n    "FieldList":[\n      "NCTId"\n    ],\n    "StudyFields":[\n      {\n        "Rank":1,\n        "NCTId":[\n          "NCT04318756"\n        ]\n      },{\n        "Rank":2,\n        "NCTId":[\n          "NCT03581708"\n        ]\n      },{\n        "Rank":3,\n        "NCTId":[\n          "NCT04834570"\n        ]\n      },{\n        "Rank":4,\n        "NCTId":[\n          "NCT00897650"\n        ]\n      },{\n        "Rank":5,\n        "NCTId":[\n          "NCT04494620"\n        ]\n      },{\n        "Rank":6,\n        "NCTId":[\n          "NCT02890667"\n        ]\n      },{\n        "Rank":7,\n        "NCTId":[\n          "NCT04875351"\n        ]\n      },{\n        "Rank":8,\n        "NCTId":[\n          "NCT00512551"\n        ]\n      },{\

In [ ]:
df_result = pd.read_json(response.content)
df_result

,StudyFieldsResponse
APIVrs,1.01.05
DataVrs,2022:01:31 23:02:41.433
Expression,cancer
FieldList,[NCTId]
MaxRank,20
MinRank,1
NStudiesAvail,402977
NStudiesFound,98915
NStudiesReturned,20
StudyFields,"[{'Rank': 1, 'NCTId': ['NCT04318756']}, {'Rank..."


In [ ]:
end_limit = df_result.loc['NStudiesFound', 'StudyFieldsResponse']
end_limit

98915

In [ ]:
ss = gc.create("Criteria-List-inclusion-check-8","1rSPoezkw1yRVb9YKeJPZe9IuOF9YaMTT")

In [ ]:
fields = "NCTId,EligibilityCriteria"
# fields = "InterventionType,ArmGroupInterventionName"
# column_headings = ["NCT ID", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Disease Characteristics"]
# column_headings = ["Interventions","Drug"]
column_headings = ["NCT ID", "Eligibility Criteria", "Inclusion Criteria"]

def create_spreadsheet(min_value, max_value):
  inclusion_criteria_count = 0
  inclusion_bracket_count = 0
  inclusion_dash_count = 0
  # disease_count = 0
  inclusion_not_found = 0
  non_inclusion_found = 0
  not_found_list = []
  result_list = []
  final_url = BASE_URL + f"{expression}&fields={fields}&min_rnk={min_value}&max_rnk={max_value}&fmt={format}" 
  print(final_url)
  response = requests.get(final_url)
  print(response)
  df_json = pd.read_json(response.content)
  df_json['StudyFieldsResponse']['StudyFields']
  for data in df_json['StudyFieldsResponse']['StudyFields']:
    # print(data)
    trial_rows=[]
    
    nct_id = ','.join(data["NCTId"])


    criteria = " ".join(data['EligibilityCriteria'])

    criteria_data = data['EligibilityCriteria']
    # print(criteria_data)

    for data in criteria_data:
      data = data.lower()
      
      if "non-inclusion" in data:
        data = data.replace("non-inclusion", "exclusion")
        non_inclusion_found += 1
      else:
        data = data
        
      if "inclusion (" in data:

        print("Inclusion (", "found")
        inclusion_count_with_colon = data.split("\n").count("inclusion:")
        inclusion_count_without_colon = data.split("\n").count("inclusion")
        inclusion_criteria_list = []
        if inclusion_count_with_colon > 1 or inclusion_count_without_colon > 1:
          print("Loop method working")
          if inclusion_count_with_colon > 1:
            inclusion_data= data.split("inclusion")
            for inclusion in inclusion_data:
  
              if "exclusion criteria:" in inclusion:
        #         print("1 worked")
                inc_end = inclusion.index("exclusion criteria:") #exclusion criteria:
              elif "exclusion criteria" in inclusion:
        #         print("2 worked")
                inc_end = inclusion.index("exclusion criteria")
              elif "exclusion" in inclusion:
                inc_end = inclusion.index("exclusion")
              elif "exclusion:" in inclusion:
                inc_end = inclusion.index("exclusion:")
              else:
        #         print("3 worked")
                inc_end = len(inclusion)
              inclusion_criteria = inclusion[:inc_end]
              inclusion_criteria_list.append(inclusion_criteria)
          elif inclusion_count_without_colon > 1:
            print("without colon worked")
            inclusion_data= data.split("inclusion")
        #     # print("GK FOUND DATA")
        #     # print(inclusion_data)
            
            for inclusion in inclusion_data:
        #       # print(inclusion)
              if "exclusion criteria:" in inclusion:
                print("1 worked")
                inc_end = inclusion.index("exclusion criteria:") #exclusion criteria:
              elif "exclusion criteria" in inclusion:
                print("2 worked")
                inc_end = inclusion.index("exclusion criteria")
              elif "exclusion" in inclusion:
                inc_end = inclusion.index("exclusion")
              elif "exclusion:" in inclusion:
                inc_end = inclusion.index("exclusion:")
              else:
                print("3 worked")
                inc_end = len(inclusion)
              inclusion_criteria = inclusion[:inc_end]
              inclusion_criteria_list.append(inclusion_criteria)
          inclusion_criteria_string = " ".join(inclusion_criteria_list)
          print(inclusion_criteria_string)
        else:
          print("Non loop method working")  
          inclusion_start = data.find("inclusion (")+len("inclusion (")
          if "exclusion" in data: 
            inclusion_end = data.find("exclusion")
          elif "exclusion:" in data:
            inclusion_end = data.find("exclusion:")
          elif "inclusion" in data:
            inclusion_end = data.find('inclusion')
          else:
            inclusion_end = len(data)
          inclusion_criteria_string = data[inclusion_start:inclusion_end]

          inclusion_bracket_count += 1

  
      elif "inclusion criteria" in data:
        print("Inclusion Criteria", "found")
        inclusion_count_with_colon = data.split("\n").count("inclusion criteria:")
        inclusion_count_without_colon = data.split("\n").count("inclusion criteria")
        inclusion_criteria_list = []
        if inclusion_count_with_colon > 1 or inclusion_count_without_colon > 1:
          print("Loop method working")
          if inclusion_count_with_colon > 1:
            inclusion_data= data.split("inclusion criteria:")
            for inclusion in inclusion_data:
  
              if "exclusion criteria:" in inclusion:
        #         print("1 worked")
                inc_end = inclusion.index("exclusion criteria:") #exclusion criteria:
              elif "exclusion criteria" in inclusion:
        #         print("2 worked")
                inc_end = inclusion.index("exclusion criteria")
              elif "exclusion" in inclusion:
                inc_end = inclusion.index("exclusion")
              elif "exclusion:" in inclusion:
                inc_end = inclusion.index("exclusion:")
              else:
        #         print("3 worked")
                inc_end = len(inclusion)
              inclusion_criteria = inclusion[:inc_end]
              inclusion_criteria_list.append(inclusion_criteria)
          elif inclusion_count_without_colon > 1:
            print("without colon worked")
            inclusion_data= data.split("inclusion criteria")
        #     # print("GK FOUND DATA")
        #     # print(inclusion_data)
            
            for inclusion in inclusion_data:
        #       # print(inclusion)
              if "exclusion criteria:" in inclusion:
                print("1 worked")
                inc_end = inclusion.index("exclusion criteria:") #exclusion criteria:
              elif "exclusion criteria" in inclusion:
                print("2 worked")
                inc_end = inclusion.index("exclusion criteria")
              elif "exclusion" in inclusion:
                inc_end = inclusion.index("exclusion")
              elif "exclusion:" in inclusion:
                inc_end = inclusion.index("exclusion:")
              else:
                print("3 worked")
                inc_end = len(inclusion)
              inclusion_criteria = inclusion[:inc_end]
              inclusion_criteria_list.append(inclusion_criteria)
          inclusion_criteria_string = " ".join(inclusion_criteria_list)
          print(inclusion_criteria_string)
        else:
          print("non loop method working")
          data = data.replace("inclusion criteria", "inclusion_criteria") #waste
          print("----------------------------------------------------------------------------------------------------------------------------------")
          print(data)
          inclusion_start = data.find("inclusion_criteria")+len("inclusion criteria:")+1
          if "exclusion" in data:
            inclusion_end = data.find("exclusion")
          elif "exclusion:" in data:
            inclusion_end = data.find("exclusion:")
          elif "inclusion" in data:
            inclusion_end = data.find('inclusion')
          
          else:
            inclusion_end = len(data)
          inclusion_criteria_string = data[inclusion_start:inclusion_end]
        print(inclusion_criteria_string)
        inclusion_criteria_count += 1
      else:
        not_found_list.append(data)
        # print("Not found")
        inclusion_not_found += 1
  #       print("\n")
  #       inclusion_criteria_string = ""
      
    # print(not_found_list)
  # print("non inclusion count:", "\t",non_inclusion_found)
  # print("Inclusion Bracket count:", "\t", inclusion_bracket_count)
  # print("Inclusion Criteria count:", "\t", inclusion_criteria_count)
  # print("Not count:", "\t", inclusion_not_found)
  
          
          
  #   #   # trial_rows.append(nct_id)
  #   #   # trial_rows.append(criteria)
  #   # print("Not found List")
  #   # for record in not_found_list:
  #   #   print(record)
    
    trial_rows.append(nct_id)
    trial_rows.append(criteria)
    trial_rows.append(inclusion_criteria_string)
    # trial_rows.append(disease_character_string)
    # trial_rows.append(nct_id)

    result_list.append(trial_rows)

  

  result_list_df = pd.DataFrame(result_list)
  result_list_df.columns = column_headings

  list_of_lists = result_list_df.to_numpy().tolist()
  headers = result_list_df.columns.tolist()

  data_to_write = [headers] + list_of_lists
  
  sh = gc.open("Criteria-List-inclusion-check-8")
  worksheet = sh.add_worksheet(title=f"sheet-new-final", rows="1002", cols="10")
  worksheet.update("A1",data_to_write,value_input_option="USER_ENTERED")
    

  

In [ ]:
min_value = 1
max_value = 1000

create_spreadsheet(2001, 3000)

# while max_value <= 5000:

#   create_spreadsheet(min_value, max_value)

#   print("done")
  
#   min_value += 1000
#   max_value += 1000
# if max_value != end_limit:
#   min_value = min_value
#   max_value = end_limit

#   create_spreadsheet(min_value, max_value)

In [ ]:
ss = gc.create("Sample_CT_LIST","13GpPvOZsZGstaK4H3O__jPzf5CU-mnrA")

In [ ]:
fields = "NCTId,EligibilityCriteria"
# fields = "InterventionType,ArmGroupInterventionName"
# column_headings = ["NCT ID", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Disease Characteristics"]
# column_headings = ["Interventions","Drug"]
column_headings = ["NCT ID", "Eligibility Criteria", "Exclusion Criteria"]

def create_spreadsheet(min_value, max_value):
  exclusion_criteria_count = 0
  exclusion_not_found_count = 0
  exclusion_colon_found = 0
  not_found_list = []
  result_list = []
  final_url = BASE_URL + f"{expression}&fields={fields}&min_rnk={min_value}&max_rnk={max_value}&fmt={format}" 
  print(final_url)
  response = requests.get(final_url)
  print(response)
  df_json = pd.read_json(response.content)
  df_json['StudyFieldsResponse']['StudyFields']
  for data in df_json['StudyFieldsResponse']['StudyFields']:
    # print(data)
    trial_rows=[]
    
    nct_id = ','.join(data["NCTId"])
    if nct_id == "NCT01648166":
      print("found")

    criteria = " ".join(data['EligibilityCriteria'])
    # print(criteria)

    criteria_data = data['EligibilityCriteria']
    print(criteria_data)

    for data in criteria_data:
      # print(type(data))
      data = data.lower()
      # print(data)
      if "non-inclusion" in data:
        data = data.replace("non-inclusion", "exclusion")
      else:
        data = data

      if "exclusion criteria" in data:
        print("Exclusion Criteria", "found")
        exclusion_count_with_colon = data.split("\n").count("exclusion criteria:")
        exclusion_count_without_colon = data.split("\n").count("exclusion criteria")
        print(exclusion_count_with_colon)
        print(exclusion_count_without_colon)
        exclusion_criteria_list = []
        if exclusion_count_with_colon > 1 or exclusion_count_without_colon > 1:
          print("Loop method working")
          if exclusion_count_with_colon > 1:
            exclusion_data= data.split("exclusion criteria:")[1:]
            # print("GK FOUND DATA")
            # print(inclusion_data)
            
            for exclusion in exclusion_data:
              # print(inclusion)
              if "inclusion criteria:" in exclusion:
                print("1 worked")
                exc_end = exclusion.index("inclusion criteria:") #exclusion criteria:
              elif "inclusion criteria" in exclusion:
                print("2 worked")
                exc_end = exclusion.index("inclusion criteria")
              else:
                print("3 worked")
                exc_end = len(exclusion)
              exclusion_criteria = exclusion[:exc_end]
              exclusion_criteria_list.append(exclusion_criteria)
          elif exclusion_count_without_colon > 1:
            print("without colon worked")
            exclusion_data= data.split("exclusion criteria")[1:]
      
            for exclusion in exclusion_data:
              # print(inclusion)
              if "inclusion criteria:" in exclusion:
                print("1 worked")
                exc_end = exclusion.index("inclusion criteria:") #exclusion criteria:
              elif "inclusion criteria" in exclusion:
                print("2 worked")
                exc_end = exclusion.index("inclusion criteria")
              else:
                print("3 worked")
                exc_end = len(exclusion)
              exclusion_criteria = exclusion[:exc_end]
              exclusion_criteria_list.append(exclusion_criteria)
          exclusion_criteria_string = " ".join(exclusion_criteria_list)
          print(exclusion_criteria_string)
        else:
          print("non loop method working") #Exclusion Criteria
          # data.replace("Exclusion Criteria:", "exclusion_criteria")
          # data.replace("Exclusion Criteria", "exclusion_criteria")

          # print("Exclusion GK DATA")
          # print(data)
          exclusion_start = data.find("exclusion")+len("exclusion criteria:")+1
          if "exclusion" in data:
            exclusion_start = data.find("exclusion")+len("exclusion criteria:")+1
          elif "exclusion:" in data:
            exclusion_start = data.find("Exclusion:")+len("exclusion:")+1
          print("EXCLUSION STARTTTTTTTTTT:", exclusion_start)
          if nct_id == "NCT01648166":
            print("*********************************")
            print(exclusion_start)
          remaining_data_for_exclusion = data.split("exclusion")[1:]
          remaining_data_for_exclusion_string = " ".join(remaining_data_for_exclusion)
          if "exclusion" in remaining_data_for_exclusion_string:
            exclusion_end = remaining_data_for_exclusion_string.find("exclusion")
          elif "inclusion" in remaining_data_for_exclusion_string:
            exclusion_end = remaining_data_for_exclusion_string.find('inclusion')
          else:
            exclusion_end = len(data)
          exclusion_criteria_string = data[exclusion_start:exclusion_end]
        exclusion_criteria_count += 1
      elif "exclusion:" in data:
        exclusion_start = data.find("exclusion:")+len("exclusion:")+1
        
        remaining_data_for_exclusion = data.split("exclusion")[1:]
        remaining_data_for_exclusion_string = " ".join(remaining_data_for_exclusion)
        if "exclusion" in remaining_data_for_exclusion_string:
          exclusion_end = remaining_data_for_exclusion_string.find("exclusion")
        elif "inclusion" in remaining_data_for_exclusion_string:
          exclusion_end = remaining_data_for_exclusion_string.find('inclusion')
        else:
          exclusion_end = len(data)
        exclusion_criteria_string = data[exclusion_start:exclusion_end]
        print("exclusion gkkgkkgkgkgkgkgkkgkgg django found")
        exclusion_colon_found += 1
      else:
        not_found_list.append(data)
        print("Not found")
        print("\n")
        exclusion_criteria_string = ""
        exclusion_not_found_count += 1
      
     
    trial_rows.append(nct_id)
    trial_rows.append(criteria)
    trial_rows.append(exclusion_criteria_string)
    
    result_list.append(trial_rows)
  # print("Exclusion Criteria count:", "\t", exclusion_criteria_count)
  # print("Exclusion Criteria not found count:", "\t", exclusion_not_found_count)   
  # print("Exclusion colon count:", "\t", exclusion_colon_found)   
  result_list_df = pd.DataFrame(result_list)
  result_list_df.columns = column_headings

  list_of_lists = result_list_df.to_numpy().tolist()
  headers = result_list_df.columns.tolist()

  data_to_write = [headers] + list_of_lists
  
  sh = gc.open("Criteria-List-exclusion-check-9")
  worksheet = sh.add_worksheet(title="sheet-new-2", rows="1002", cols="10")
  worksheet.update("A1",data_to_write,value_input_option="USER_ENTERED")
    

In [ ]:
min_value = 1
max_value = 1000

create_spreadsheet(2001, 3000)


Streaming output truncated to the last 5000 lines.
["Inclusion Criteria:\n\nWomen who satisfy all of the following conditions are the only patients who will be eligible for this study:\n\nPatient must have invasive ductal, medullary, papillary, colloid (mucinous), or tubular histologies.\nAJCC Stage I or IIA (T1N0 or T2N0) histologically confirmed invasive carcinoma of the breast with a primary lesion (less than/equal to 3 cm)by MRI or ultrasound\nClinically negative axillary lymph nodes. Standard routine imaging assessment of the axilla is performed by either ultrasound or MRI. If lymph nodes are suspicious, a biopsy is required. An involved axilla is purposely excluded.\nUnifocal breast cancer (single focus which can be encompassed by one lumpectomy).\nPatients must have an estrogen receptor (ER), progesterone receptor (PR), and human epidermal growth receptor 2 (H2N) analyses performed on the primary tumor prior to enrollment.\nBased on pre-treatment planning CT scan, the ratio of t